# Data Cleaning
## NYC Open *Big* Data Analysis
Author: Mark Bauer

Objective: Clean data to use for analysis.

In [1]:
# import libraries
import pandas as pd

In [2]:
# list files in directory
%ls

LICENSE              data/                log.txt
README.md            data-cleaning.ipynb
analysis.ipynb       data-export.ipynb


In [3]:
# log file
file = 'log.txt'

# manually specify column names
names = [
    'datetime_log',
    'id',
    'error_log',
    'count_rows'
]

# read log file into dataframe
df = pd.read_csv(file, names=names)

# preview data
print(f"shape of data: {df.shape}")
df.head()

shape of data: (2554, 4)


,datetime_log,id,error_log,count_rows
0,2024-08-09 13:56:42,fkec-mjr6,NaN,182.0
1,2024-08-09 13:56:47,mzxg-pwib,NaN,27673.0
2,2024-08-09 13:56:53,6r9j-qrwz,NaN,91.0
3,2024-08-09 13:57:00,99xv-he3n,NaN,188.0
4,2024-08-09 13:57:06,ufxk-pq9j,NaN,39.0


In [4]:
# column information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2554 entries, 0 to 2553
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   datetime_log  2554 non-null   object 
 1   id            2554 non-null   object 
 2   error_log     1 non-null      object 
 3   count_rows    2553 non-null   float64
dtypes: float64(1), object(3)
memory usage: 79.9+ KB


There's one record with a non-null error log.

In [5]:
# summary statistics
df.describe().round(1)

,count_rows
count,2553.0
mean,2337140.8
std,18089982.8
min,0.0
25%,138.0
50%,1396.0
75%,12383.0
max,376404531.0


In [6]:
# is dataset id unique
df['id'].is_unique

True

In [7]:
# count nulls per column
df.isnull().sum()

datetime_log       0
id                 0
error_log       2553
count_rows         1
dtype: int64

Check row with null `count_rows`.

In [8]:
df.loc[df['count_rows'].isnull()]

,datetime_log,id,error_log,count_rows
174,2024-08-09 14:32:04,erdf-2akx,Request error for erdf-2akx: 408 Client Error:...,NaN


In [9]:
# preview error log
df.loc[df['error_log'].notnull(), 'error_log'].values[0]

'Request error for erdf-2akx: 408 Client Error: Request Timeout for url: https://data.cityofnewyork.us/resource/erdf-2akx.json?$select=count(*)'

I reproduced this error given the https address above. The request timed out. Skip this dataset for now.

In [10]:
# fill null count_rows and cast column as int
df['count_rows'] = df['count_rows'].fillna(0).astype(int)

df.head()

,datetime_log,id,error_log,count_rows
0,2024-08-09 13:56:42,fkec-mjr6,NaN,182
1,2024-08-09 13:56:47,mzxg-pwib,NaN,27673
2,2024-08-09 13:56:53,6r9j-qrwz,NaN,91
3,2024-08-09 13:57:00,99xv-he3n,NaN,188
4,2024-08-09 13:57:06,ufxk-pq9j,NaN,39


# Metadata API
Learn more about the Socrata Metadata API on the official docs here: https://dev.socrata.com/docs/other/metadata#?route=overview.

In [11]:
# read dataset metadata to join information
path = 'https://data.cityofnewyork.us/api/views/metadata/v1'
metadata_df = pd.read_json(path)

print(metadata_df.shape)
metadata_df.head()

(3235, 21)


,id,name,attribution,attributionLink,category,createdAt,dataUpdatedAt,dataUri,description,domain,...,hideFromCatalog,hideFromDataJson,license,metadataUpdatedAt,provenance,updatedAt,webUri,approvals,customFields,tags
0,fkec-mjr6,"DOHMH Cryptosporidiosis by Race/Ethnicity, Age...",Department of Health and Mental Hygiene (DOHMH),None,Health,2024-08-05T14:12:47+0000,2024-08-05T16:04:46+0000,https://data.cityofnewyork.us/resource/fkec-mjr6,"Cryptosporidiosis, number of cases and annual ...",data.cityofnewyork.us,...,False,False,None,2024-08-05T16:33:29+0000,OFFICIAL,2024-08-05T16:34:05+0000,https://data.cityofnewyork.us/d/fkec-mjr6,"[{'reviewedAt': 1722875645, 'reviewedAutomatic...","{'Update': {'Automation': 'No', 'Date Made Pub...","[cryptosporidiosis, diagnosis year, race ethni..."
1,r6e8-2fwe,Location of Disposal Facilities and Sites Used...,NYC Department of Sanitation (DSNY),None,City Government,2024-07-31T14:38:56+0000,2024-07-31T14:33:03+0000,https://data.cityofnewyork.us/resource/r6e8-2fwe,The location of the disposal facilities where ...,data.cityofnewyork.us,...,False,False,None,2024-07-31T19:40:30+0000,OFFICIAL,2024-07-31T19:53:25+0000,https://data.cityofnewyork.us/d/r6e8-2fwe,"[{'reviewedAt': 1722455605, 'reviewedAutomatic...",{'Data Collection': {'Data Collection': 'Dispo...,None
2,9e2b-mctv,New York City Bike Routes\t (Map),Department of Transportation (DOT),https://www.nyc.gov/html/dot/html/bicyclists/b...,None,2024-07-24T16:08:52+0000,2024-07-24T16:06:04+0000,https://data.cityofnewyork.us/resource/9e2b-mctv,The New York City Department of Transportation...,data.cityofnewyork.us,...,False,False,None,2024-08-06T21:34:51+0000,OFFICIAL,2024-08-06T21:34:51+0000,https://data.cityofnewyork.us/d/9e2b-mctv,"[{'reviewedAt': 1722300713, 'reviewedAutomatic...","{'Update': {'Automation': 'No', 'Update Freque...",None
3,mzxg-pwib,New York City Bike Routes,Department of Transportation (DOT),https://www.nyc.gov/html/dot/html/bicyclists/b...,None,2024-07-24T15:57:31+0000,2024-07-24T16:06:04+0000,https://data.cityofnewyork.us/resource/mzxg-pwib,The New York City Department of Transportation...,data.cityofnewyork.us,...,False,False,None,2024-07-30T00:50:54+0000,OFFICIAL,2024-07-30T00:51:27+0000,https://data.cityofnewyork.us/d/mzxg-pwib,"[{'reviewedAt': 1722300687, 'reviewedAutomatic...","{'Update': {'Automation': 'No', 'Date Made Pub...","[nyc bike routes, bike routes]"
4,6r9j-qrwz,DSNY Disposal Facilities Used by Year,NYC Department of Sanitation (DSNY),None,City Government,2024-07-12T18:37:24+0000,2024-07-31T14:21:50+0000,https://data.cityofnewyork.us/resource/6r9j-qrwz,A listing of the facilities used by year to ha...,data.cityofnewyork.us,...,False,False,None,2024-07-31T19:45:38+0000,OFFICIAL,2024-07-31T19:51:22+0000,https://data.cityofnewyork.us/d/6r9j-qrwz,"[{'reviewedAt': 1722455482, 'reviewedAutomatic...",{'Data Collection': {'Data Collection': 'Dispo...,"[sanitation, waste, transfer station, waste to..."


In [12]:
# metadata column info
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3235 entries, 0 to 3234
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 3235 non-null   object 
 1   name               3235 non-null   object 
 2   attribution        3092 non-null   object 
 3   attributionLink    455 non-null    object 
 4   category           3125 non-null   object 
 5   createdAt          3235 non-null   object 
 6   dataUpdatedAt      3062 non-null   object 
 7   dataUri            3235 non-null   object 
 8   description        3161 non-null   object 
 9   domain             3235 non-null   object 
 10  externalId         0 non-null      float64
 11  hideFromCatalog    3235 non-null   bool   
 12  hideFromDataJson   3235 non-null   bool   
 13  license            79 non-null     object 
 14  metadataUpdatedAt  3235 non-null   object 
 15  provenance         3235 non-null   object 
 16  updatedAt          3235 

# Discovery API
Similar to the Metadata API but contains much more information on how the datasets are being used on NYC Open Data.

In [13]:
# discovery views
path = 'https://data.cityofnewyork.us/api/views/'
views_df = pd.read_json(path)

print(views_df.shape)
views_df.head()

(3235, 50)


,id,name,assetType,averageRating,category,createdAt,description,displayType,downloadCount,hideFromCatalog,...,blobFilename,blobFileSize,blobId,blobMimeType,ratings,childViews,indexUpdatedAt,iconUrl,previewImageId,disabledFeatureFlags
0,fkec-mjr6,"DOHMH Cryptosporidiosis by Race/Ethnicity, Age...",dataset,0,Health,1722867167,"Cryptosporidiosis, number of cases and annual ...",table,4,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,r6e8-2fwe,Location of Disposal Facilities and Sites Used...,map,0,City Government,1722436736,The location of the disposal facilities where ...,visualization_canvas_map,0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9e2b-mctv,New York City Bike Routes\t (Map),map,0,NaN,1721837332,The New York City Department of Transportation...,visualization_canvas_map,0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mzxg-pwib,New York City Bike Routes,dataset,0,NaN,1721836651,The New York City Department of Transportation...,table,101,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6r9j-qrwz,DSNY Disposal Facilities Used by Year,dataset,0,City Government,1720809444,A listing of the facilities used by year to ha...,table,7,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# preview column info
views_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3235 entries, 0 to 3234
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        3235 non-null   object 
 1   name                      3235 non-null   object 
 2   assetType                 3235 non-null   object 
 3   averageRating             3235 non-null   int64  
 4   category                  3125 non-null   object 
 5   createdAt                 3235 non-null   int64  
 6   description               3161 non-null   object 
 7   displayType               3235 non-null   object 
 8   downloadCount             3235 non-null   int64  
 9   hideFromCatalog           3235 non-null   bool   
 10  hideFromDataJson          3235 non-null   bool   
 11  locked                    3235 non-null   bool   
 12  newBackend                3235 non-null   bool   
 13  numberOfComments          3235 non-null   int64  
 14  oid     

In [15]:
# retrieve only selected columns
cols = [
    'id',
    'viewCount', 'downloadCount',
    'assetType', 'displayType'
]

views_df = views_df.loc[:, cols]

views_df.head()

,id,viewCount,downloadCount,assetType,displayType
0,fkec-mjr6,84,4,dataset,table
1,r6e8-2fwe,55,0,map,visualization_canvas_map
2,9e2b-mctv,343,0,map,visualization_canvas_map
3,mzxg-pwib,471,101,dataset,table
4,6r9j-qrwz,96,7,dataset,table


## Merge Metadata and Discovery APIs together

In [16]:
# merge metadata with metadata views
metadata_merged_df = metadata_df.merge(
    views_df,
    on='id',
    how='right'
)

print(metadata_merged_df.shape)
metadata_merged_df.head()

(3235, 25)


,id,name,attribution,attributionLink,category,createdAt,dataUpdatedAt,dataUri,description,domain,...,provenance,updatedAt,webUri,approvals,customFields,tags,viewCount,downloadCount,assetType,displayType
0,fkec-mjr6,"DOHMH Cryptosporidiosis by Race/Ethnicity, Age...",Department of Health and Mental Hygiene (DOHMH),None,Health,2024-08-05T14:12:47+0000,2024-08-05T16:04:46+0000,https://data.cityofnewyork.us/resource/fkec-mjr6,"Cryptosporidiosis, number of cases and annual ...",data.cityofnewyork.us,...,OFFICIAL,2024-08-05T16:34:05+0000,https://data.cityofnewyork.us/d/fkec-mjr6,"[{'reviewedAt': 1722875645, 'reviewedAutomatic...","{'Update': {'Automation': 'No', 'Date Made Pub...","[cryptosporidiosis, diagnosis year, race ethni...",84,4,dataset,table
1,r6e8-2fwe,Location of Disposal Facilities and Sites Used...,NYC Department of Sanitation (DSNY),None,City Government,2024-07-31T14:38:56+0000,2024-07-31T14:33:03+0000,https://data.cityofnewyork.us/resource/r6e8-2fwe,The location of the disposal facilities where ...,data.cityofnewyork.us,...,OFFICIAL,2024-07-31T19:53:25+0000,https://data.cityofnewyork.us/d/r6e8-2fwe,"[{'reviewedAt': 1722455605, 'reviewedAutomatic...",{'Data Collection': {'Data Collection': 'Dispo...,None,55,0,map,visualization_canvas_map
2,9e2b-mctv,New York City Bike Routes\t (Map),Department of Transportation (DOT),https://www.nyc.gov/html/dot/html/bicyclists/b...,None,2024-07-24T16:08:52+0000,2024-07-24T16:06:04+0000,https://data.cityofnewyork.us/resource/9e2b-mctv,The New York City Department of Transportation...,data.cityofnewyork.us,...,OFFICIAL,2024-08-06T21:34:51+0000,https://data.cityofnewyork.us/d/9e2b-mctv,"[{'reviewedAt': 1722300713, 'reviewedAutomatic...","{'Update': {'Automation': 'No', 'Update Freque...",None,343,0,map,visualization_canvas_map
3,mzxg-pwib,New York City Bike Routes,Department of Transportation (DOT),https://www.nyc.gov/html/dot/html/bicyclists/b...,None,2024-07-24T15:57:31+0000,2024-07-24T16:06:04+0000,https://data.cityofnewyork.us/resource/mzxg-pwib,The New York City Department of Transportation...,data.cityofnewyork.us,...,OFFICIAL,2024-07-30T00:51:27+0000,https://data.cityofnewyork.us/d/mzxg-pwib,"[{'reviewedAt': 1722300687, 'reviewedAutomatic...","{'Update': {'Automation': 'No', 'Date Made Pub...","[nyc bike routes, bike routes]",471,101,dataset,table
4,6r9j-qrwz,DSNY Disposal Facilities Used by Year,NYC Department of Sanitation (DSNY),None,City Government,2024-07-12T18:37:24+0000,2024-07-31T14:21:50+0000,https://data.cityofnewyork.us/resource/6r9j-qrwz,A listing of the facilities used by year to ha...,data.cityofnewyork.us,...,OFFICIAL,2024-07-31T19:51:22+0000,https://data.cityofnewyork.us/d/6r9j-qrwz,"[{'reviewedAt': 1722455482, 'reviewedAutomatic...",{'Data Collection': {'Data Collection': 'Dispo...,"[sanitation, waste, transfer station, waste to...",96,7,dataset,table


In [17]:
# preview column info
metadata_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3235 entries, 0 to 3234
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 3235 non-null   object 
 1   name               3235 non-null   object 
 2   attribution        3092 non-null   object 
 3   attributionLink    455 non-null    object 
 4   category           3125 non-null   object 
 5   createdAt          3235 non-null   object 
 6   dataUpdatedAt      3062 non-null   object 
 7   dataUri            3235 non-null   object 
 8   description        3161 non-null   object 
 9   domain             3235 non-null   object 
 10  externalId         0 non-null      float64
 11  hideFromCatalog    3235 non-null   bool   
 12  hideFromDataJson   3235 non-null   bool   
 13  license            79 non-null     object 
 14  metadataUpdatedAt  3235 non-null   object 
 15  provenance         3235 non-null   object 
 16  updatedAt          3235 

In [18]:
# select specific columns
cols = [
    'id', 'name', 'attribution', 'description',
    'viewCount', 'downloadCount',
    'category', 'assetType', 'displayType', 'tags',
    'createdAt', 'updatedAt', 'dataUpdatedAt', 'metadataUpdatedAt',
    'domain', 'attributionLink', 'webUri', 'dataUri'
]

metadata_merged_df = metadata_merged_df.loc[:, cols]

metadata_merged_df.head()

,id,name,attribution,description,viewCount,downloadCount,category,assetType,displayType,tags,createdAt,updatedAt,dataUpdatedAt,metadataUpdatedAt,domain,attributionLink,webUri,dataUri
0,fkec-mjr6,"DOHMH Cryptosporidiosis by Race/Ethnicity, Age...",Department of Health and Mental Hygiene (DOHMH),"Cryptosporidiosis, number of cases and annual ...",84,4,Health,dataset,table,"[cryptosporidiosis, diagnosis year, race ethni...",2024-08-05T14:12:47+0000,2024-08-05T16:34:05+0000,2024-08-05T16:04:46+0000,2024-08-05T16:33:29+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/fkec-mjr6,https://data.cityofnewyork.us/resource/fkec-mjr6
1,r6e8-2fwe,Location of Disposal Facilities and Sites Used...,NYC Department of Sanitation (DSNY),The location of the disposal facilities where ...,55,0,City Government,map,visualization_canvas_map,None,2024-07-31T14:38:56+0000,2024-07-31T19:53:25+0000,2024-07-31T14:33:03+0000,2024-07-31T19:40:30+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/r6e8-2fwe,https://data.cityofnewyork.us/resource/r6e8-2fwe
2,9e2b-mctv,New York City Bike Routes\t (Map),Department of Transportation (DOT),The New York City Department of Transportation...,343,0,None,map,visualization_canvas_map,None,2024-07-24T16:08:52+0000,2024-08-06T21:34:51+0000,2024-07-24T16:06:04+0000,2024-08-06T21:34:51+0000,data.cityofnewyork.us,https://www.nyc.gov/html/dot/html/bicyclists/b...,https://data.cityofnewyork.us/d/9e2b-mctv,https://data.cityofnewyork.us/resource/9e2b-mctv
3,mzxg-pwib,New York City Bike Routes,Department of Transportation (DOT),The New York City Department of Transportation...,471,101,None,dataset,table,"[nyc bike routes, bike routes]",2024-07-24T15:57:31+0000,2024-07-30T00:51:27+0000,2024-07-24T16:06:04+0000,2024-07-30T00:50:54+0000,data.cityofnewyork.us,https://www.nyc.gov/html/dot/html/bicyclists/b...,https://data.cityofnewyork.us/d/mzxg-pwib,https://data.cityofnewyork.us/resource/mzxg-pwib
4,6r9j-qrwz,DSNY Disposal Facilities Used by Year,NYC Department of Sanitation (DSNY),A listing of the facilities used by year to ha...,96,7,City Government,dataset,table,"[sanitation, waste, transfer station, waste to...",2024-07-12T18:37:24+0000,2024-07-31T19:51:22+0000,2024-07-31T14:21:50+0000,2024-07-31T19:45:38+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/6r9j-qrwz,https://data.cityofnewyork.us/resource/6r9j-qrwz


In [19]:
# we only want datasets that are datasets (e.g. not map or dashboards) and are displayed as tables
metadata_merged_df = metadata_merged_df.loc[
    (metadata_merged_df['assetType'] == 'dataset')
    & (metadata_merged_df['displayType'] == 'table')
]

print(metadata_merged_df.shape)
metadata_merged_df.head()

(2552, 18)


,id,name,attribution,description,viewCount,downloadCount,category,assetType,displayType,tags,createdAt,updatedAt,dataUpdatedAt,metadataUpdatedAt,domain,attributionLink,webUri,dataUri
0,fkec-mjr6,"DOHMH Cryptosporidiosis by Race/Ethnicity, Age...",Department of Health and Mental Hygiene (DOHMH),"Cryptosporidiosis, number of cases and annual ...",84,4,Health,dataset,table,"[cryptosporidiosis, diagnosis year, race ethni...",2024-08-05T14:12:47+0000,2024-08-05T16:34:05+0000,2024-08-05T16:04:46+0000,2024-08-05T16:33:29+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/fkec-mjr6,https://data.cityofnewyork.us/resource/fkec-mjr6
3,mzxg-pwib,New York City Bike Routes,Department of Transportation (DOT),The New York City Department of Transportation...,471,101,None,dataset,table,"[nyc bike routes, bike routes]",2024-07-24T15:57:31+0000,2024-07-30T00:51:27+0000,2024-07-24T16:06:04+0000,2024-07-30T00:50:54+0000,data.cityofnewyork.us,https://www.nyc.gov/html/dot/html/bicyclists/b...,https://data.cityofnewyork.us/d/mzxg-pwib,https://data.cityofnewyork.us/resource/mzxg-pwib
4,6r9j-qrwz,DSNY Disposal Facilities Used by Year,NYC Department of Sanitation (DSNY),A listing of the facilities used by year to ha...,96,7,City Government,dataset,table,"[sanitation, waste, transfer station, waste to...",2024-07-12T18:37:24+0000,2024-07-31T19:51:22+0000,2024-07-31T14:21:50+0000,2024-07-31T19:45:38+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/6r9j-qrwz,https://data.cityofnewyork.us/resource/6r9j-qrwz
5,99xv-he3n,DSNY Disposal Sites Used by Facilities by Year,NYC Department of Sanitation (DSNY),A listing of the disposal sites used by each f...,71,8,City Government,dataset,table,"[sanitation, waste, transfer station, waste to...",2024-07-12T18:18:59+0000,2024-07-31T19:51:26+0000,2024-07-31T14:18:13+0000,2024-07-31T19:44:47+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/99xv-he3n,https://data.cityofnewyork.us/resource/99xv-he3n
6,ufxk-pq9j,Location of Disposal Facilities and Sites Used...,NYC Department of Sanitation (DSNY),The location of the disposal facilities where ...,84,17,City Government,dataset,table,"[sanitation, waste, transfer station, waste to...",2024-07-12T17:54:05+0000,2024-07-31T19:51:24+0000,2024-07-31T14:33:03+0000,2024-07-31T19:45:15+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/ufxk-pq9j,https://data.cityofnewyork.us/resource/ufxk-pq9j


In [20]:
# now we can safely drop these columns, as each value is the same
cols = ['assetType', 'displayType']
metadata_merged_df = metadata_merged_df.loc[
    :,
    ~metadata_merged_df.columns.isin(cols)
]

metadata_merged_df.head()

,id,name,attribution,description,viewCount,downloadCount,category,tags,createdAt,updatedAt,dataUpdatedAt,metadataUpdatedAt,domain,attributionLink,webUri,dataUri
0,fkec-mjr6,"DOHMH Cryptosporidiosis by Race/Ethnicity, Age...",Department of Health and Mental Hygiene (DOHMH),"Cryptosporidiosis, number of cases and annual ...",84,4,Health,"[cryptosporidiosis, diagnosis year, race ethni...",2024-08-05T14:12:47+0000,2024-08-05T16:34:05+0000,2024-08-05T16:04:46+0000,2024-08-05T16:33:29+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/fkec-mjr6,https://data.cityofnewyork.us/resource/fkec-mjr6
3,mzxg-pwib,New York City Bike Routes,Department of Transportation (DOT),The New York City Department of Transportation...,471,101,None,"[nyc bike routes, bike routes]",2024-07-24T15:57:31+0000,2024-07-30T00:51:27+0000,2024-07-24T16:06:04+0000,2024-07-30T00:50:54+0000,data.cityofnewyork.us,https://www.nyc.gov/html/dot/html/bicyclists/b...,https://data.cityofnewyork.us/d/mzxg-pwib,https://data.cityofnewyork.us/resource/mzxg-pwib
4,6r9j-qrwz,DSNY Disposal Facilities Used by Year,NYC Department of Sanitation (DSNY),A listing of the facilities used by year to ha...,96,7,City Government,"[sanitation, waste, transfer station, waste to...",2024-07-12T18:37:24+0000,2024-07-31T19:51:22+0000,2024-07-31T14:21:50+0000,2024-07-31T19:45:38+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/6r9j-qrwz,https://data.cityofnewyork.us/resource/6r9j-qrwz
5,99xv-he3n,DSNY Disposal Sites Used by Facilities by Year,NYC Department of Sanitation (DSNY),A listing of the disposal sites used by each f...,71,8,City Government,"[sanitation, waste, transfer station, waste to...",2024-07-12T18:18:59+0000,2024-07-31T19:51:26+0000,2024-07-31T14:18:13+0000,2024-07-31T19:44:47+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/99xv-he3n,https://data.cityofnewyork.us/resource/99xv-he3n
6,ufxk-pq9j,Location of Disposal Facilities and Sites Used...,NYC Department of Sanitation (DSNY),The location of the disposal facilities where ...,84,17,City Government,"[sanitation, waste, transfer station, waste to...",2024-07-12T17:54:05+0000,2024-07-31T19:51:24+0000,2024-07-31T14:33:03+0000,2024-07-31T19:45:15+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/ufxk-pq9j,https://data.cityofnewyork.us/resource/ufxk-pq9j


In [21]:
# merge dataset log file with metadata
merged_df = df.merge(
    metadata_merged_df,
    on='id',
    how='left'
)

print(merged_df.shape)
merged_df.head()

(2554, 19)


,datetime_log,id,error_log,count_rows,name,attribution,description,viewCount,downloadCount,category,tags,createdAt,updatedAt,dataUpdatedAt,metadataUpdatedAt,domain,attributionLink,webUri,dataUri
0,2024-08-09 13:56:42,fkec-mjr6,NaN,182,"DOHMH Cryptosporidiosis by Race/Ethnicity, Age...",Department of Health and Mental Hygiene (DOHMH),"Cryptosporidiosis, number of cases and annual ...",84.0,4.0,Health,"[cryptosporidiosis, diagnosis year, race ethni...",2024-08-05T14:12:47+0000,2024-08-05T16:34:05+0000,2024-08-05T16:04:46+0000,2024-08-05T16:33:29+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/fkec-mjr6,https://data.cityofnewyork.us/resource/fkec-mjr6
1,2024-08-09 13:56:47,mzxg-pwib,NaN,27673,New York City Bike Routes,Department of Transportation (DOT),The New York City Department of Transportation...,471.0,101.0,None,"[nyc bike routes, bike routes]",2024-07-24T15:57:31+0000,2024-07-30T00:51:27+0000,2024-07-24T16:06:04+0000,2024-07-30T00:50:54+0000,data.cityofnewyork.us,https://www.nyc.gov/html/dot/html/bicyclists/b...,https://data.cityofnewyork.us/d/mzxg-pwib,https://data.cityofnewyork.us/resource/mzxg-pwib
2,2024-08-09 13:56:53,6r9j-qrwz,NaN,91,DSNY Disposal Facilities Used by Year,NYC Department of Sanitation (DSNY),A listing of the facilities used by year to ha...,96.0,7.0,City Government,"[sanitation, waste, transfer station, waste to...",2024-07-12T18:37:24+0000,2024-07-31T19:51:22+0000,2024-07-31T14:21:50+0000,2024-07-31T19:45:38+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/6r9j-qrwz,https://data.cityofnewyork.us/resource/6r9j-qrwz
3,2024-08-09 13:57:00,99xv-he3n,NaN,188,DSNY Disposal Sites Used by Facilities by Year,NYC Department of Sanitation (DSNY),A listing of the disposal sites used by each f...,71.0,8.0,City Government,"[sanitation, waste, transfer station, waste to...",2024-07-12T18:18:59+0000,2024-07-31T19:51:26+0000,2024-07-31T14:18:13+0000,2024-07-31T19:44:47+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/99xv-he3n,https://data.cityofnewyork.us/resource/99xv-he3n
4,2024-08-09 13:57:06,ufxk-pq9j,NaN,39,Location of Disposal Facilities and Sites Used...,NYC Department of Sanitation (DSNY),The location of the disposal facilities where ...,84.0,17.0,City Government,"[sanitation, waste, transfer station, waste to...",2024-07-12T17:54:05+0000,2024-07-31T19:51:24+0000,2024-07-31T14:33:03+0000,2024-07-31T19:45:15+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/ufxk-pq9j,https://data.cityofnewyork.us/resource/ufxk-pq9j


In [22]:
# rearrange columns
cols = [
    'id', 'name', 'attribution', 'description',
    'count_rows', 'viewCount', 'downloadCount',
     'category', 'tags',
    'createdAt', 'updatedAt', 'dataUpdatedAt', 'metadataUpdatedAt',
    'domain', 'attributionLink', 'webUri', 'dataUri'
]

merged_df = merged_df.loc[:, cols]

merged_df.head()

,id,name,attribution,description,count_rows,viewCount,downloadCount,category,tags,createdAt,updatedAt,dataUpdatedAt,metadataUpdatedAt,domain,attributionLink,webUri,dataUri
0,fkec-mjr6,"DOHMH Cryptosporidiosis by Race/Ethnicity, Age...",Department of Health and Mental Hygiene (DOHMH),"Cryptosporidiosis, number of cases and annual ...",182,84.0,4.0,Health,"[cryptosporidiosis, diagnosis year, race ethni...",2024-08-05T14:12:47+0000,2024-08-05T16:34:05+0000,2024-08-05T16:04:46+0000,2024-08-05T16:33:29+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/fkec-mjr6,https://data.cityofnewyork.us/resource/fkec-mjr6
1,mzxg-pwib,New York City Bike Routes,Department of Transportation (DOT),The New York City Department of Transportation...,27673,471.0,101.0,None,"[nyc bike routes, bike routes]",2024-07-24T15:57:31+0000,2024-07-30T00:51:27+0000,2024-07-24T16:06:04+0000,2024-07-30T00:50:54+0000,data.cityofnewyork.us,https://www.nyc.gov/html/dot/html/bicyclists/b...,https://data.cityofnewyork.us/d/mzxg-pwib,https://data.cityofnewyork.us/resource/mzxg-pwib
2,6r9j-qrwz,DSNY Disposal Facilities Used by Year,NYC Department of Sanitation (DSNY),A listing of the facilities used by year to ha...,91,96.0,7.0,City Government,"[sanitation, waste, transfer station, waste to...",2024-07-12T18:37:24+0000,2024-07-31T19:51:22+0000,2024-07-31T14:21:50+0000,2024-07-31T19:45:38+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/6r9j-qrwz,https://data.cityofnewyork.us/resource/6r9j-qrwz
3,99xv-he3n,DSNY Disposal Sites Used by Facilities by Year,NYC Department of Sanitation (DSNY),A listing of the disposal sites used by each f...,188,71.0,8.0,City Government,"[sanitation, waste, transfer station, waste to...",2024-07-12T18:18:59+0000,2024-07-31T19:51:26+0000,2024-07-31T14:18:13+0000,2024-07-31T19:44:47+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/99xv-he3n,https://data.cityofnewyork.us/resource/99xv-he3n
4,ufxk-pq9j,Location of Disposal Facilities and Sites Used...,NYC Department of Sanitation (DSNY),The location of the disposal facilities where ...,39,84.0,17.0,City Government,"[sanitation, waste, transfer station, waste to...",2024-07-12T17:54:05+0000,2024-07-31T19:51:24+0000,2024-07-31T14:33:03+0000,2024-07-31T19:45:15+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/ufxk-pq9j,https://data.cityofnewyork.us/resource/ufxk-pq9j


In [23]:
# preview column info
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2554 entries, 0 to 2553
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2554 non-null   object 
 1   name               2551 non-null   object 
 2   attribution        2425 non-null   object 
 3   description        2477 non-null   object 
 4   count_rows         2554 non-null   int64  
 5   viewCount          2551 non-null   float64
 6   downloadCount      2551 non-null   float64
 7   category           2464 non-null   object 
 8   tags               1903 non-null   object 
 9   createdAt          2551 non-null   object 
 10  updatedAt          2551 non-null   object 
 11  dataUpdatedAt      2531 non-null   object 
 12  metadataUpdatedAt  2551 non-null   object 
 13  domain             2551 non-null   object 
 14  attributionLink    358 non-null    object 
 15  webUri             2551 non-null   object 
 16  dataUri            2551 

In [24]:
# summary statistics
merged_df.describe().round(1)

,count_rows,viewCount,downloadCount
count,2554.0,2551.0,2551.0
mean,2336225.7,10864.0,4509.6
std,18086498.7,107109.0,39693.1
min,0.0,49.0,4.0
25%,135.8,361.5,388.0
50%,1395.0,822.0,784.0
75%,12382.8,2536.5,2079.0
max,376404531.0,2806198.0,1612110.0


In [25]:
# null counts per column
merged_df.isnull().sum()

id                      0
name                    3
attribution           129
description            77
count_rows              0
viewCount               3
downloadCount           3
category               90
tags                  651
createdAt               3
updatedAt               3
dataUpdatedAt          23
metadataUpdatedAt       3
domain                  3
attributionLink      2196
webUri                  3
dataUri                 3
dtype: int64

Examine why three datasets have `name` as null.

In [26]:
merged_df.loc[merged_df['name'].isnull()]

,id,name,attribution,description,count_rows,viewCount,downloadCount,category,tags,createdAt,updatedAt,dataUpdatedAt,metadataUpdatedAt,domain,attributionLink,webUri,dataUri
358,in83-58q5,NaN,NaN,NaN,334044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,evu4-6zyr,NaN,NaN,NaN,335616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360,njuk-taxk,NaN,NaN,NaN,309528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
merged_df.loc[merged_df['name'].isnull()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 358 to 360
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 3 non-null      object 
 1   name               0 non-null      object 
 2   attribution        0 non-null      object 
 3   description        0 non-null      object 
 4   count_rows         3 non-null      int64  
 5   viewCount          0 non-null      float64
 6   downloadCount      0 non-null      float64
 7   category           0 non-null      object 
 8   tags               0 non-null      object 
 9   createdAt          0 non-null      object 
 10  updatedAt          0 non-null      object 
 11  dataUpdatedAt      0 non-null      object 
 12  metadataUpdatedAt  0 non-null      object 
 13  domain             0 non-null      object 
 14  attributionLink    0 non-null      object 
 15  webUri             0 non-null      object 
 16  dataUri            0 non-n

These might be unauthorized or private datasets hosted on NYC Open Data. I couldn't access them. Let's drop them.

In [28]:
merged_df = (
    merged_df
    .loc[merged_df['name'].notnull()]
    .reset_index(drop=True)
)

print(merged_df.shape)
merged_df.head()

(2551, 17)


,id,name,attribution,description,count_rows,viewCount,downloadCount,category,tags,createdAt,updatedAt,dataUpdatedAt,metadataUpdatedAt,domain,attributionLink,webUri,dataUri
0,fkec-mjr6,"DOHMH Cryptosporidiosis by Race/Ethnicity, Age...",Department of Health and Mental Hygiene (DOHMH),"Cryptosporidiosis, number of cases and annual ...",182,84.0,4.0,Health,"[cryptosporidiosis, diagnosis year, race ethni...",2024-08-05T14:12:47+0000,2024-08-05T16:34:05+0000,2024-08-05T16:04:46+0000,2024-08-05T16:33:29+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/fkec-mjr6,https://data.cityofnewyork.us/resource/fkec-mjr6
1,mzxg-pwib,New York City Bike Routes,Department of Transportation (DOT),The New York City Department of Transportation...,27673,471.0,101.0,None,"[nyc bike routes, bike routes]",2024-07-24T15:57:31+0000,2024-07-30T00:51:27+0000,2024-07-24T16:06:04+0000,2024-07-30T00:50:54+0000,data.cityofnewyork.us,https://www.nyc.gov/html/dot/html/bicyclists/b...,https://data.cityofnewyork.us/d/mzxg-pwib,https://data.cityofnewyork.us/resource/mzxg-pwib
2,6r9j-qrwz,DSNY Disposal Facilities Used by Year,NYC Department of Sanitation (DSNY),A listing of the facilities used by year to ha...,91,96.0,7.0,City Government,"[sanitation, waste, transfer station, waste to...",2024-07-12T18:37:24+0000,2024-07-31T19:51:22+0000,2024-07-31T14:21:50+0000,2024-07-31T19:45:38+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/6r9j-qrwz,https://data.cityofnewyork.us/resource/6r9j-qrwz
3,99xv-he3n,DSNY Disposal Sites Used by Facilities by Year,NYC Department of Sanitation (DSNY),A listing of the disposal sites used by each f...,188,71.0,8.0,City Government,"[sanitation, waste, transfer station, waste to...",2024-07-12T18:18:59+0000,2024-07-31T19:51:26+0000,2024-07-31T14:18:13+0000,2024-07-31T19:44:47+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/99xv-he3n,https://data.cityofnewyork.us/resource/99xv-he3n
4,ufxk-pq9j,Location of Disposal Facilities and Sites Used...,NYC Department of Sanitation (DSNY),The location of the disposal facilities where ...,39,84.0,17.0,City Government,"[sanitation, waste, transfer station, waste to...",2024-07-12T17:54:05+0000,2024-07-31T19:51:24+0000,2024-07-31T14:33:03+0000,2024-07-31T19:45:15+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/ufxk-pq9j,https://data.cityofnewyork.us/resource/ufxk-pq9j


In [29]:
# sanity check
merged_df.isnull().sum()

id                      0
name                    0
attribution           126
description            74
count_rows              0
viewCount               0
downloadCount           0
category               87
tags                  648
createdAt               0
updatedAt               0
dataUpdatedAt          20
metadataUpdatedAt       0
domain                  0
attributionLink      2193
webUri                  0
dataUri                 0
dtype: int64

In [30]:
# cast desired columns to int
merged_df = merged_df.astype({
    'viewCount':int,
    'downloadCount':int
})

merged_df.head()

,id,name,attribution,description,count_rows,viewCount,downloadCount,category,tags,createdAt,updatedAt,dataUpdatedAt,metadataUpdatedAt,domain,attributionLink,webUri,dataUri
0,fkec-mjr6,"DOHMH Cryptosporidiosis by Race/Ethnicity, Age...",Department of Health and Mental Hygiene (DOHMH),"Cryptosporidiosis, number of cases and annual ...",182,84,4,Health,"[cryptosporidiosis, diagnosis year, race ethni...",2024-08-05T14:12:47+0000,2024-08-05T16:34:05+0000,2024-08-05T16:04:46+0000,2024-08-05T16:33:29+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/fkec-mjr6,https://data.cityofnewyork.us/resource/fkec-mjr6
1,mzxg-pwib,New York City Bike Routes,Department of Transportation (DOT),The New York City Department of Transportation...,27673,471,101,None,"[nyc bike routes, bike routes]",2024-07-24T15:57:31+0000,2024-07-30T00:51:27+0000,2024-07-24T16:06:04+0000,2024-07-30T00:50:54+0000,data.cityofnewyork.us,https://www.nyc.gov/html/dot/html/bicyclists/b...,https://data.cityofnewyork.us/d/mzxg-pwib,https://data.cityofnewyork.us/resource/mzxg-pwib
2,6r9j-qrwz,DSNY Disposal Facilities Used by Year,NYC Department of Sanitation (DSNY),A listing of the facilities used by year to ha...,91,96,7,City Government,"[sanitation, waste, transfer station, waste to...",2024-07-12T18:37:24+0000,2024-07-31T19:51:22+0000,2024-07-31T14:21:50+0000,2024-07-31T19:45:38+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/6r9j-qrwz,https://data.cityofnewyork.us/resource/6r9j-qrwz
3,99xv-he3n,DSNY Disposal Sites Used by Facilities by Year,NYC Department of Sanitation (DSNY),A listing of the disposal sites used by each f...,188,71,8,City Government,"[sanitation, waste, transfer station, waste to...",2024-07-12T18:18:59+0000,2024-07-31T19:51:26+0000,2024-07-31T14:18:13+0000,2024-07-31T19:44:47+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/99xv-he3n,https://data.cityofnewyork.us/resource/99xv-he3n
4,ufxk-pq9j,Location of Disposal Facilities and Sites Used...,NYC Department of Sanitation (DSNY),The location of the disposal facilities where ...,39,84,17,City Government,"[sanitation, waste, transfer station, waste to...",2024-07-12T17:54:05+0000,2024-07-31T19:51:24+0000,2024-07-31T14:33:03+0000,2024-07-31T19:45:15+0000,data.cityofnewyork.us,None,https://data.cityofnewyork.us/d/ufxk-pq9j,https://data.cityofnewyork.us/resource/ufxk-pq9j


In [31]:
# save cleaned dataset
merged_df.to_csv('data/data.csv', index=False)

In [32]:
%ls data/

data.csv
